In [ ]:
import os
from typing import Optional, TypedDict, Annotated
from langchain_core.documents.base import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool
import warnings
import dotenv

warnings.filterwarnings("ignore")

In [2]:
# .env파일 읽기
dotenv.load_dotenv()

# os라이브러리 이용한 읽기
# os.environ['OPENAI_API_KEY'] = "API-Key"
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

True

In [3]:
# LLM Load

llm = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0.)

In [51]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
    """

    context : {context}
    
    당신은 언제나 고객에게 최선을 다해 답변을 하며 말투는 굉장히 친근합니다. 직업은 전문 상담원입니다. 답변 시, 아래의 규칙을 지켜야만 합니다.
    규칙 1. 주어진 context만을 이용하여 답변해야합니다. 
    규칙 2. 주어진 context에서 답변을 할 수 없다면 "해당 문의는 010-2255-3366으로 연락주시면 도와드리겠습니다. 영업 시간은 오전 10시-오후 6시입니다." 라고 대답하세요.
    규칙 3. 문자열에 A1, A2, A11, A22 등 필요 없는 문자는 제거한 뒤 출력합니다.
    규칙 4. 항상 친절한 말투로 응대합니다.
    규칙 5. 웹사이트 링크를 그대로 출력합니다. 대소문자를 명확하게 구분하세요.
    """),
    ("human", "{query}")
])

In [44]:
# Document Embedding

def embedding_file(file="Practice_document.pdf"):
    print("Embedding 시작")
    Embeddings = OpenAIEmbeddings()
    print("Embedding 완료!\n파일 로딩중")
    loader = UnstructuredFileLoader(file_path=f"./files/{file}")
    print("파일 로딩 완료!\n텍스트 스플리터 생성")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=["\n\n"])
    print("문서 분할 시작")
    split_docs = loader.load_and_split(text_splitter)
    print("문서 분할 완료")
    vector_store = FAISS.from_documents(split_docs, Embeddings)
    print("벡터 저장소 생성 완료")

    retriever = vector_store.as_retriever()
    print("벡터 리트리버 생성 완료")

    return retriever

In [21]:
retriever = embedding_file()

Embedding 시작
Embedding 완료!
파일 로딩중
파일 로딩 완료!
텍스트 스플리터 생성
문서 분할 시작작
문서 분할 완료
벡터 저장소 생성 완료
벡터 리트리버 생성 완료


In [26]:
retriever.invoke("수강료는 얼마인가요?")

[Document(id='c3311b5c-1aee-4e65-86e7-bb87390d11c8', metadata={'source': './files/Practice_document.pdf'}, page_content='모두의연구소 역삼 캠퍼스 (서울 강남구 역삼로 156 태광빌딩 2층, 4층) https://naver.me/GMvc9Hv5\n\nFAQ\n\n[지원 요건] Q1. 재직중(또는 재학중)인데 지원할 수 있는지 궁금해요!\n\nA1. 아이펠 교육과정은 약 6개월 동안 월-금, 10:00-18:00 풀타임으로 운영되고 있어 직장 또는 학교와 병행이 어렵습니다. 내일배움카드 발급이 가능한 구직자 및 대학교 졸업 예정자를 주요 대상자로 모집, 선발하고 있으니 참여 가능 사항을 꼭 확인해주세요!'),
 Document(id='d3beb99c-0571-4797-8227-de2339aaeba6', metadata={'source': './files/Practice_document.pdf'}, page_content='2\n\nQ17. 서류를 제출 했어요. 결과는 언제 알 수 있나요?\n\nA17. 제출하신 지원서는 검토 후 2-3일 이내에 결과를 안내하고 있습니다. ‘지원서 합격 → 온라인 인터뷰 → 최종 선발ʼ의 절차로 진행되고 있어 최종 합격 및 등록까지 일정 기간 소요될 수 있으니 조금만 기다려주세요!\n\n[훈련비] Q18. 등록금(훈련비)은 정말 무료인가요?\n\nA18. 내일배움카드 발급이 가능하다면 100% 무료로 입학이 가능합니다. 단, 잔액이 1원이라도 있어야 하니 꼭 확인해주세요!'),
 Document(id='5d91b849-b866-4298-ab5b-067b4df72dd2', metadata={'source': './files/Practice_document.pdf'}, page_content='Q12. 학습량이 많은가요?\n\nA12. 학습량이 적다고 말씀드리기는 어렵습니다. 비전공자부터 전공자까지 모두 배움의 정도에 차이가 있고 상당한 수

In [49]:
chain = {"context":retriever | RunnableLambda(lambda docs:'\n\n'.join(x.page_content for x in docs)),
         "query":RunnablePassthrough()}| prompt | llm

In [50]:
result = chain.invoke("수강료는 얼마인가요?")

ValueError: variable context should be a list of base messages, got 모두의연구소 역삼 캠퍼스 (서울 강남구 역삼로 156 태광빌딩 2층, 4층) https://naver.me/GMvc9Hv5

FAQ

[지원 요건] Q1. 재직중(또는 재학중)인데 지원할 수 있는지 궁금해요!

A1. 아이펠 교육과정은 약 6개월 동안 월-금, 10:00-18:00 풀타임으로 운영되고 있어 직장 또는 학교와 병행이 어렵습니다. 내일배움카드 발급이 가능한 구직자 및 대학교 졸업 예정자를 주요 대상자로 모집, 선발하고 있으니 참여 가능 사항을 꼭 확인해주세요!

2

Q17. 서류를 제출 했어요. 결과는 언제 알 수 있나요?

A17. 제출하신 지원서는 검토 후 2-3일 이내에 결과를 안내하고 있습니다. ‘지원서 합격 → 온라인 인터뷰 → 최종 선발ʼ의 절차로 진행되고 있어 최종 합격 및 등록까지 일정 기간 소요될 수 있으니 조금만 기다려주세요!

[훈련비] Q18. 등록금(훈련비)은 정말 무료인가요?

A18. 내일배움카드 발급이 가능하다면 100% 무료로 입학이 가능합니다. 단, 잔액이 1원이라도 있어야 하니 꼭 확인해주세요!

Q12. 학습량이 많은가요?

A12. 학습량이 적다고 말씀드리기는 어렵습니다. 비전공자부터 전공자까지 모두 배움의 정도에 차이가 있고 상당한 수준까지 학습이 진행되기에 개인 공부가 거의 필수로 작용할 수 있습니다. 다만, 이러한 과정을 지나고나면 매우 성장해있는 본인의 모습을 보실 수 있으실겁니다. 화이팅!

[훈련 환경] Q13. 교육 중 카메라를 꼭 켜야하나요?

[학습 장소] Q8. 과정은 모두 온라인으로만 진행 되나요?

A8. 아이펠 교육과정은 100% 실시간 온라인 과정으로 화상 플랫폼(ZEP)에서 진행됩니다. 희망하시는 경우, 오프라인 학습 공간(모두의연구소 강남/역삼캠퍼스)을 무료로 제공 해드리고 있습니다. 공간 사용방법은 입학 후 별도로 안내드리겠습니다.

[학습 방식] Q9. 혼자

In [18]:
result

AIMessage(content='내일배움카드 발급이 가능하다면 100% 무료로 입학이 가능합니다. 단, 잔액이 1원이라도 있어야 하니 꼭 확인해주세요! 더 궁금한 점이 있으시면 언제든지 문의해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 775, 'total_tokens': 832, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4916fb9-3d2f-46b5-b954-681226b08222-0', usage_metadata={'input_tokens': 775, 'output_tokens': 57, 'total_tokens': 832, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})